## Agent
Agent：这是一个类，负责决定下一步要采取什么行动。它由语言模型和提示驱动。提示可以包括agent的性格、背景上下文以及用于引发更好推理的提示策略等。
Tools：这些是agent调用的函数。这里有两个重要的考虑因素：给agent提供正确的工具，以及以最有帮助的方式描述这些工具。
Toolkits：这是一组工具，用于完成特定的目标。LangChain提供了一系列的toolkits。
AgentExecutor：这是agent的运行时环境。它实际上调用agent并执行其选择的动作。

下面是完整的代码：

In [1]:
from langchain.agents import OpenAIFunctionsAgent
# 定义用来控制Agent的模型
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

# 定义一个Tool，这个Tool是用来计算单单词的长度
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """返回单词的长度。"""
    return len(word)


tools = [get_word_length]

# 创建提示，设定Agent的特点和擅长点
from langchain.schema import SystemMessage

system_message = SystemMessage(content="你是一个非常强大的助手，但是在计算单词长度方面不擅长。")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

# 定义Agent
from langchain.agents import OpenAIFunctionsAgent

agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

# 创建AgentExecutor - 我们的agent的运行时环境。
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [5]:
# 测试一下
agent_executor.run("单词educa有多少个字母？")
agent_executor.run("那是个真正的单词吗？")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5单词"educa"有5个字母。

> Finished chain.


> Entering new AgentExecutor chain...
请提供一个单词，我将告诉您它是否是一个真正的单词。

> Finished chain.


'请提供一个单词，我将告诉您它是否是一个真正的单词。'

In [6]:
from langchain.prompts import MessagesPlaceholder
# 给Agent加上会话状态
from langchain.memory import ConversationBufferMemory

MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

In [7]:
# 加上状态Memory
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
agent_executor.run("单词educa有多少个字母？")
agent_executor.run("那是个真正的单词吗？")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5单词"educa"有5个字母。

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5是的，"educa"是一个真正的单词，它有5个字母。

> Finished chain.


'是的，"educa"是一个真正的单词，它有5个字母。'

In [8]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='单词educa有多少个字母？', additional_kwargs={}, example=False),
  AIMessage(content='单词"educa"有5个字母。', additional_kwargs={}, example=False),
  HumanMessage(content='那是个真正的单词吗？', additional_kwargs={}, example=False),
  AIMessage(content='是的，"educa"是一个真正的单词，它有5个字母。', additional_kwargs={}, example=False)]}

## AgentTypes

### Conversational
使用Agent来实现聊天

In [9]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

search = SerpAPIWrapper()
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="当你需要回答关于当前事件或世界当前状态的问题时很有用"
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

llm = OpenAI(temperature=0)
agent_chain = initialize_agent(
    # 工具列表
    tools,
    llm,
    # Agent类型
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory)

In [10]:
agent_chain.run(input="你好，我是艾贺")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: 你好，很高兴认识你！

> Finished chain.


'你好，很高兴认识你！'

In [11]:
agent_chain.run(input="我的名字是什么?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: 你的名字是艾贺！

> Finished chain.


'你的名字是艾贺！'

In [12]:
agent_chain.run("我想买一件衬衫，能给一些建议吗?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: 当然可以！我建议你可以去看看当地的服装店，或者在网上搜索一些衬衫的品牌，看看有没有你喜欢的款式。

> Finished chain.


'当然可以！我建议你可以去看看当地的服装店，或者在网上搜索一些衬衫的品牌，看看有没有你喜欢的款式。'

In [13]:
agent_chain.run(input="明天杭州的天气怎么样? 针对明天的天气我适合买什么样的衣服")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: 明天杭州的天气
Observation: 23日（今天）. 雷阵雨. 30 ; 24日（明天）. 雷阵雨转晴. 32 ; 25日（后天）. 多云. 35 ; 26日（周三）. 雷阵雨转多云. 33 ; 27日（周四）. 小到中雨转阴. 31 ...
Thought: Do I need to use a tool? No
AI: 根据杭州明天的天气预报，明天会有雷阵雨，建议你买一些防水的衣服，比如雨衣、雨鞋等，以及一些厚实的衣服，比如棉衣、毛衣等，以防止受凉。

> Finished chain.


'根据杭州明天的天气预报，明天会有雷阵雨，建议你买一些防水的衣服，比如雨衣、雨鞋等，以及一些厚实的衣服，比如棉衣、毛衣等，以防止受凉。'

In [14]:
memory.load_memory_variables({})

{'chat_history': 'Human: 你好，我是艾贺\nAI: 你好，很高兴认识你！\nHuman: 我的名字是什么?\nAI: 你的名字是艾贺！\nHuman: 我想买一件衬衫，能给一些建议吗?\nAI: 当然可以！我建议你可以去看看当地的服装店，或者在网上搜索一些衬衫的品牌，看看有没有你喜欢的款式。\nHuman: 明天杭州的天气怎么样? 针对明天的天气我适合买什么样的衣服\nAI: 根据杭州明天的天气预报，明天会有雷阵雨，建议你买一些防水的衣服，比如雨衣、雨鞋等，以及一些厚实的衣服，比如棉衣、毛衣等，以防止受凉。'}

### OpenAI 函数
某些OpenAI模型（如gpt-3.5-turbo-0613和gpt-4-0613）已经被微调
在API调用中，你可以描述函数，并让模型智能地选择输出一个包含调用这些函数的参数的JSON对象。
OpenAI函数API的目标是比通用文本完成或聊天API更可靠地返回有效且有用的函数调用。

In [15]:
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [16]:
agent.run("特朗普今年多大? 他的年龄再除以2为多少？")



> Entering new AgentExecutor chain...

Invoking: `Search` with `特朗普年龄`


77 years
Invoking: `Calculator` with `77 / 2`




> Entering new LLMMathChain chain...
77 / 2```text
77 / 2
```
...numexpr.evaluate("77 / 2")...

Answer: 38.5
> Finished chain.
Answer: 38.5特朗普今年77岁，他的年龄除以2等于38.5岁。

> Finished chain.


'特朗普今年77岁，他的年龄除以2等于38.5岁。'

### Self ask with search


In [17]:
# 导入所需的模块
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

# 初始化OpenAI模型，设置温度参数为0
llm = OpenAI(temperature=0)

# 初始化SerpAPIWrapper，用于进行网络搜索
search = SerpAPIWrapper()

# 定义工具列表，其中包含一个名为"Intermediate Answer"的工具，该工具使用search.run函数进行搜索
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="当你需要进行搜索提问时很有用",
    )
]

# 初始化代理，设置代理类型为SELF_ASK_WITH_SEARCH，开启详细模式
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 阿里巴巴的财报是什么时候发布的？

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [18]:
# 运行代理，提问"现任男子美国公开赛冠军的家乡是哪里？"
self_ask_with_search.run(
    "阿里巴巴最近一次的财报营业额和利润是多少？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 阿里巴巴的财报是什么时候发布的？
Intermediate answer: 5月18日，阿里巴巴发布2023财年第四季度及全年财报。 公司财报整体优于预期，2023财年第四季度实现净利润219.96亿元，上年同期亏损为183.57亿元，公司商业板块明显复苏。
Follow up: 阿里巴巴最近一次的财报营业额和利润是多少？
Intermediate answer: 营业利润, 15240, 37745, 25137, 24943. 利息收入（开支）- 非营运净额, -, -, -, -. 出售资产收入（亏损）, -, -, -42452, -. 其他，净额, -10514, -11821, 87484 ...
So the final answer is: 营业利润, 15240, 37745, 25137, 24943. 利息收入（开支）- 非营运净额, -, -, -, -. 出售资产收入（亏损）, -, -, -42452, -. 其他，净额, -10514, -11821, 87484 ...

> Finished chain.


'营业利润, 15240, 37745, 25137, 24943. 利息收入（开支）- 非营运净额, -, -, -, -. 出售资产收入（亏损）, -, -, -42452, -. 其他，净额, -10514, -11821, 87484 ...'

## Structured tool chat
可以使用args_schema计算动作的输入，约定输入的结构。在使用structured-chat-zero-shot-react-description 或者 AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION是有用的。

In [2]:
import requests
from pydantic import BaseModel, Field
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent
from langchain.tools import StructuredTool



def create_crowd(type: str, param: dict) -> str:
    """
     该工具可以用来进行人群生成：
    当需要生成人群、分析画像、咨询问题时，使用如下的指示：url 固定为：http://localhost:3001/
     如果请求是生成人群，请求的type为crowd; 如果请求是分析画像，请求的type为analyze; 如果是其他或者答疑，请求的type为question;
     请求body的param把用户指定的条件传进来即可
     只要请求有结果，你就说人群正在生成中就行
     """
    result = requests.post("http://localhost:3001/", json={"type": type, "param": param})
    print(result)
    return f"Status: {result.status_code} - {result.text}"

tools = [
    StructuredTool.from_function(func=create_crowd, return_direct=True)
]

llm = OpenAI(temperature=0)  # Also works well with Anthropic models
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_chain = initialize_agent(tools,
                               llm,
                               agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True,
                               # memory=memory
                               )

In [3]:
agent_chain.run("我想生成一个性别为男并且在180天访问过淘特的人群?")



> Entering new AgentExecutor chain...

Action:
```
{
  "action": "create_crowd",
  "action_input": {
    "type": "crowd",
    "param": {
      "gender": "male",
      "visit_taote": "180"
    }
  }
}
```



ConnectionError: HTTPConnectionPool(host='localhost', port=3001): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x133c8ee20>: Failed to establish a new connection: [Errno 61] Connection refused'))